<a href="https://colab.research.google.com/github/rsricha8/RecommenderSystemsML/blob/main/RecommenderSystems_collaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
movies_df = pd.read_csv('dataset/movies.csv')
links_df = pd.read_csv('dataset/links.csv')
ratings_df = pd.read_csv('dataset/ratings.csv')
tags_df = pd.read_csv('dataset/tags.csv')

print('The dimensions of movies are:', movies_df.shape, 'The dimensions of ratings are:', ratings_df.shape,)

The dimensions of movies are: (9742, 3) The dimensions of ratings are: (100836, 4)


In [8]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
#movie id to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of user:", n_users,"and number of movies:", n_items, ". Hence full rating matrix will be:",n_users*n_items)
print("Number of ratings available:", len(ratings_df), ". Percentage of matrix filled:", len(ratings_df)/(n_users*n_items) *100)

Number of user: 610 and number of movies: 9724 . Hence full rating matrix will be: 5931640
Number of ratings available: 100836 . Percentage of matrix filled: 1.6999683055613624


In [17]:
import torch
import numpy as np
from torch.autograd import Variable

class MatrixFactorization(torch.nn.Module):
  def __init__(self, n_users, n_items, n_factors=20):
    super().__init__()
    #create user embeddings
    self.user_factors = torch.nn.Embeddings(n_users, n_factors)
    #create item embeddings
    self.item_factors = torch.nn.Embeddings(n_items, n_factors)
    self.user_factors.weight.data.uniform(0, 0.05)
    self.item_factors.weight.data.uniform(0, 0.05)

  def forward(self, data):
    #matrix multiplication
    users, items = data[:,0],data[:,1]
    return (self.user_factors(users) * self.item_factors(items)).sum(1)

  def predict(self, user, item):
    return self.forward(user, item)